In [ ]:
## This file implements neural networks with/without dropout and regularizer for p003ppresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p003ppresabsSTCC_qual.csv')
df.shape

(253, 1093)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,...,group_8643,group_8644,group_8645,group_8646,group_8815,group_8892,group_9489,ST,CC,pheno
0,107,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,5,5,0


In [6]:
df['pheno'].value_counts()

0    224
1     26
2      3
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 1092)

In [9]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTGAAAATAATCATTAGCTTGCTCACTATATAATTTGATGAATATATTTCGTGAAAGTGGGTATTTATTTAATGATTATTCTATATATGATAGTATA,...,group_8643,group_8644,group_8645,group_8646,group_8815,group_8892,group_9489,ST,CC,pheno
0,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,5,5,0
1,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,5,5,0
4,1,1,1,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,5,5,0


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 1092) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 224), (1, 224), (2, 224)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,NRS265,1
1,GA984,0
2,NRS119,0
3,NRS249,1
4,NRS255,2
...,...,...
197,NRS035,1
198,NRS387,1
199,NRS222,0
200,BCH-SA-11,1


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 336us/step - loss: 6.8981 - accuracy: 0.4234 - val_loss: 2.8814 - val_accuracy: 0.5990
Epoch 2/100
470/470 [==============================] - 0s 198us/step - loss: 2.0459 - accuracy: 0.6000 - val_loss: 1.1859 - val_accuracy: 0.6238
Epoch 3/100
470/470 [==============================] - 0s 195us/step - loss: 0.8780 - accuracy: 0.6894 - val_loss: 0.7755 - val_accuracy: 0.7129
Epoch 4/100
470/470 [==============================] - 0s 204us/step - loss: 0.5573 - accuracy: 0.7702 - val_loss: 0.4714 - val_accuracy: 0.8317
Epoch 5/100
470/470 [==============================] - 0s 220us/step - loss: 0.4097 - accuracy: 0.8404 - val_loss: 0.4820 - val_accuracy: 0.8317
Epoch 6/100
470/470 [==============================] - 0s 197us/step - loss: 0.3511 - accuracy: 0.8787 - val_loss: 0.3805 - val_accuracy: 0.8366
Epoch 7/100
470/470 [==============================] - 0s 189us/step - loss: 0.2910 

Epoch 57/100
470/470 [==============================] - 0s 183us/step - loss: 0.0292 - accuracy: 0.9936 - val_loss: 0.1542 - val_accuracy: 0.9505
Epoch 58/100
470/470 [==============================] - 0s 174us/step - loss: 0.0269 - accuracy: 0.9979 - val_loss: 0.1313 - val_accuracy: 0.9604
Epoch 59/100
470/470 [==============================] - 0s 178us/step - loss: 0.0237 - accuracy: 0.9979 - val_loss: 0.1762 - val_accuracy: 0.9505
Epoch 60/100
470/470 [==============================] - 0s 198us/step - loss: 0.0244 - accuracy: 0.9979 - val_loss: 0.1168 - val_accuracy: 0.9604
Epoch 61/100
470/470 [==============================] - 0s 172us/step - loss: 0.0301 - accuracy: 0.9979 - val_loss: 0.1461 - val_accuracy: 0.9554
Epoch 62/100
470/470 [==============================] - 0s 146us/step - loss: 0.0463 - accuracy: 0.9851 - val_loss: 0.0896 - val_accuracy: 0.9752
Epoch 63/100
470/470 [==============================] - 0s 166us/step - loss: 0.0330 - accuracy: 0.9979 - val_loss: 0.1272 -

In [31]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

202/202 [==============================] - 0s 94us/step
over-sampling test accuracy: 96.53%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 0, 2, 2, 2, 0,
       0, 0, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 1, 1, 0,
       0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 1, 0, 2, 2, 0, 2, 1, 2, 2, 1, 1, 0,
       2, 1, 2, 2, 0, 0, 2, 2, 1, 1, 0, 0, 1, 2, 0, 0, 2, 1, 2, 2, 1, 1,
       1, 1, 2, 0, 2, 0, 0, 0, 2, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 2, 2, 1, 1, 2, 0, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2,
       1, 0, 2, 1, 0, 0, 2, 0, 2, 0, 2, 0, 2, 1, 2, 1, 1, 2, 1, 1, 2, 0,
       1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 1, 2, 0, 1, 1, 1, 1,
       1, 0, 1, 2])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,NRS265,1,1
1,GA984,0,0
2,NRS119,0,0
3,NRS249,1,1
4,NRS255,2,2
...,...,...,...
197,NRS035,1,1
198,NRS387,1,1
199,NRS222,0,0
200,BCH-SA-11,1,1


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,0.007024,0.991650,1.325420e-03
1,0.999941,0.000059,4.880754e-09
2,0.996793,0.003207,1.617158e-08
3,0.003590,0.996410,1.313869e-08
4,0.000304,0.000675,9.990209e-01
...,...,...,...
197,0.004205,0.993926,1.868723e-03
198,0.007822,0.992174,4.554854e-06
199,0.999939,0.000061,1.616807e-09
200,0.007896,0.992104,6.915510e-12


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p003ppST.csv", index = False,
         header=None)

In [41]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 178us/step - loss: 4.7440e-04 - accuracy: 1.0000 - val_loss: 0.1769 - val_accuracy: 0.9653
Epoch 2/100
470/470 [==============================] - 0s 132us/step - loss: 4.8139e-04 - accuracy: 1.0000 - val_loss: 0.1814 - val_accuracy: 0.9653
Epoch 3/100
470/470 [==============================] - 0s 118us/step - loss: 4.5614e-04 - accuracy: 1.0000 - val_loss: 0.1831 - val_accuracy: 0.9653
Epoch 4/100
470/470 [==============================] - 0s 117us/step - loss: 4.5420e-04 - accuracy: 1.0000 - val_loss: 0.1869 - val_accuracy: 0.9653
Epoch 5/100
470/470 [==============================] - 0s 126us/step - loss: 4.4543e-04 - accuracy: 1.0000 - val_loss: 0.1822 - val_accuracy: 0.9653
Epoch 6/100
470/470 [==============================] - 0s 118us/step - loss: 4.3990e-04 - accuracy: 1.0000 - val_loss: 0.1859 - val_accuracy: 0.9653
Epoch 7/100
470/470 [==============================] - 0s 11

In [42]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [18]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [19]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [20]:
y_prob = df_proba[df_proba['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[7.02440600e-03, 9.91650200e-01, 1.32541950e-03],
       [9.99941000e-01, 5.89633860e-05, 4.88075400e-09],
       [9.96793100e-01, 3.20686240e-03, 1.61715850e-08],
       [3.59037000e-03, 9.96409600e-01, 1.31386875e-08],
       [3.03660320e-04, 6.75374300e-04, 9.99020930e-01],
       [4.17271400e-03, 9.95708000e-01, 1.19338260e-04],
       [3.57785400e-04, 9.27944800e-01, 7.16974960e-02],
       [2.42306000e-02, 9.75754800e-01, 1.44839480e-05],
       [3.03660320e-04, 6.75374300e-04, 9.99020930e-01],
       [9.99743760e-01, 2.56290220e-04, 4.72695300e-08],
       [1.07946340e-02, 9.89205360e-01, 1.70191750e-10],
       [9.99692560e-01, 3.07434560e-04, 9.32627400e-11],
       [7.82173900e-03, 9.92173700e-01, 4.55485360e-06],
       [9.99972700e-01, 2.22440760e-05, 4.96942500e-06],
       [3.94075100e-03, 9.91183400e-01, 4.87595200e-03],
       [3.03660320e-04, 6.75374300e-04, 9.99020930e-01],
       [9.95971000e-01, 4.02895550e-03, 2.87199650e-10],
       [9.99805030e-01, 1.94940

In [21]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9921503593145383

In [23]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [24]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9921503593145383

In [25]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [26]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [27]:
dat2

,0,test
0,CFBRSa05,0
1,NRS114,0
2,NRS168,1
3,NRS255,2
4,NRS209,2
...,...,...
197,NRS196,0
198,NRS255,2
199,NRS249,1
200,NRS209,2


In [28]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [29]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 346us/step - loss: 1.1844 - accuracy: 0.5681 - val_loss: 0.6902 - val_accuracy: 0.7129
Epoch 2/100
470/470 [==============================] - 0s 142us/step - loss: 0.6260 - accuracy: 0.7532 - val_loss: 0.5320 - val_accuracy: 0.7871
Epoch 3/100
470/470 [==============================] - 0s 141us/step - loss: 0.4054 - accuracy: 0.8340 - val_loss: 0.4342 - val_accuracy: 0.8168
Epoch 4/100
470/470 [==============================] - 0s 148us/step - loss: 0.3519 - accuracy: 0.8681 - val_loss: 0.4104 - val_accuracy: 0.8267
Epoch 5/100
470/470 [==============================] - 0s 142us/step - loss: 0.3358 - accuracy: 0.8787 - val_loss: 0.3797 - val_accuracy: 0.8366
Epoch 6/100
470/470 [==============================] - 0s 147us/step - loss: 0.2898 - accuracy: 0.9106 - val_loss: 0.3103 - val_accuracy: 0.8911
Epoch 7/100
470/470 [==============================] - 0s 135us/step - loss: 0.2388 

Epoch 57/100
470/470 [==============================] - 0s 258us/step - loss: 0.0255 - accuracy: 0.9957 - val_loss: 0.1035 - val_accuracy: 0.9653
Epoch 58/100
470/470 [==============================] - 0s 300us/step - loss: 0.0215 - accuracy: 0.9957 - val_loss: 0.1471 - val_accuracy: 0.9455
Epoch 59/100
470/470 [==============================] - 0s 175us/step - loss: 0.0280 - accuracy: 0.9957 - val_loss: 0.1120 - val_accuracy: 0.9604
Epoch 60/100
470/470 [==============================] - 0s 135us/step - loss: 0.0238 - accuracy: 0.9957 - val_loss: 0.0958 - val_accuracy: 0.9653
Epoch 61/100
470/470 [==============================] - 0s 155us/step - loss: 0.0244 - accuracy: 0.9957 - val_loss: 0.1078 - val_accuracy: 0.9604
Epoch 62/100
470/470 [==============================] - 0s 140us/step - loss: 0.0241 - accuracy: 0.9957 - val_loss: 0.1240 - val_accuracy: 0.9554
Epoch 63/100
470/470 [==============================] - 0s 132us/step - loss: 0.0259 - accuracy: 0.9957 - val_loss: 0.0963 -

In [50]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

202/202 [==============================] - 0s 83us/step
over-sampling test accuracy: 94.06%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 0, 1, 2, 2, 2, 2, 1, 2, 2, 0, 1, 0, 0, 2, 1, 0, 2, 1, 2, 2, 2,
       0, 2, 0, 1, 2, 2, 2, 2, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 2, 0, 1, 0,
       2, 0, 1, 2, 2, 2, 1, 0, 0, 2, 2, 1, 1, 2, 0, 2, 0, 0, 1, 1, 1, 1,
       1, 2, 1, 2, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1, 2, 0, 0, 1, 0, 0, 1,
       1, 2, 0, 1, 2, 1, 0, 1, 0, 2, 0, 1, 1, 1, 1, 2, 2, 1, 0, 0, 1, 0,
       2, 0, 1, 1, 1, 2, 0, 2, 1, 1, 0, 2, 1, 1, 0, 1, 0, 0, 0, 2, 1, 1,
       1, 0, 1, 1, 1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2,
       0, 2, 1, 1, 0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 2, 1,
       1, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 0, 0, 1, 2, 2, 1, 0,
       2, 1, 2, 0])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,CFBRSa05,0,0
1,NRS114,0,0
2,NRS168,1,1
3,NRS255,2,2
4,NRS209,2,2
...,...,...,...
197,NRS196,0,0
198,NRS255,2,2
199,NRS249,1,1
200,NRS209,2,2


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,9.999901e-01,0.000009,4.882489e-07
1,9.996415e-01,0.000358,4.248437e-10
2,9.225085e-02,0.907734,1.547077e-05
3,3.477896e-04,0.000522,9.991297e-01
4,8.505357e-08,0.000279,9.997205e-01
...,...,...,...
197,9.984702e-01,0.001519,1.068622e-05
198,3.477892e-04,0.000522,9.991297e-01
199,7.244510e-03,0.992755,9.145550e-09
200,8.505357e-08,0.000279,9.997205e-01


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p003ppST.csv", index = False,
         header=None)

In [54]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 210us/step - loss: 0.0157 - accuracy: 0.9979 - val_loss: 0.1100 - val_accuracy: 0.9653
Epoch 2/100
470/470 [==============================] - 0s 210us/step - loss: 0.0142 - accuracy: 0.9979 - val_loss: 0.1555 - val_accuracy: 0.9554
Epoch 3/100
470/470 [==============================] - 0s 192us/step - loss: 0.0087 - accuracy: 0.9979 - val_loss: 0.1477 - val_accuracy: 0.9554
Epoch 4/100
470/470 [==============================] - 0s 169us/step - loss: 0.0094 - accuracy: 0.9979 - val_loss: 0.1452 - val_accuracy: 0.9653
Epoch 5/100
470/470 [==============================] - 0s 151us/step - loss: 0.0124 - accuracy: 0.9979 - val_loss: 0.1532 - val_accuracy: 0.9505
Epoch 6/100
470/470 [==============================] - 0s 148us/step - loss: 0.0116 - accuracy: 0.9979 - val_loss: 0.1720 - val_accuracy: 0.9455
Epoch 7/100
470/470 [==============================] - 0s 130us/step - loss: 0.0072 

Epoch 57/100
470/470 [==============================] - 0s 233us/step - loss: 0.0418 - accuracy: 0.9830 - val_loss: 0.1800 - val_accuracy: 0.9554
Epoch 58/100
470/470 [==============================] - 0s 234us/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.1400 - val_accuracy: 0.9604
Epoch 59/100
470/470 [==============================] - 0s 178us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1711 - val_accuracy: 0.9604
Epoch 60/100
470/470 [==============================] - 0s 203us/step - loss: 0.0052 - accuracy: 0.9979 - val_loss: 0.1386 - val_accuracy: 0.9604
Epoch 61/100
470/470 [==============================] - 0s 198us/step - loss: 0.0062 - accuracy: 0.9979 - val_loss: 0.1421 - val_accuracy: 0.9604
Epoch 62/100
470/470 [==============================] - 0s 228us/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.1942 - val_accuracy: 0.9554
Epoch 63/100
470/470 [==============================] - 0s 186us/step - loss: 0.0092 - accuracy: 0.9979 - val_loss: 0.1108 -

In [55]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 99.81%


In [30]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [31]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [32]:
y_prob2 = df_proba2[df_proba2['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.99990100e-01, 9.44159700e-06, 4.88248900e-07],
       [9.99641540e-01, 3.58473360e-04, 4.24843660e-10],
       [9.22508540e-02, 9.07733600e-01, 1.54707740e-05],
       [3.47789580e-04, 5.22496470e-04, 9.99129700e-01],
       [8.50535700e-08, 2.79327240e-04, 9.99720500e-01],
       [3.47789580e-04, 5.22496470e-04, 9.99129700e-01],
       [8.50535700e-08, 2.79327240e-04, 9.99720500e-01],
       [2.94126900e-05, 9.99071100e-01, 8.99392900e-04],
       [8.50535700e-08, 2.79327240e-04, 9.99720500e-01],
       [2.23307160e-04, 9.53532200e-03, 9.90241350e-01],
       [9.99989750e-01, 9.71582500e-06, 4.34336070e-07],
       [2.89969840e-03, 9.97099040e-01, 1.32851770e-06],
       [9.99998330e-01, 1.53668780e-06, 7.53934900e-08],
       [9.95054360e-01, 3.56378830e-03, 1.38182480e-03],
       [3.47789580e-04, 5.22496470e-04, 9.99129700e-01],
       [9.22508540e-02, 9.07733600e-01, 1.54707740e-05],
       [9.98790300e-01, 1.20721850e-03, 2.50849280e-06],
       [3.47789580e-04, 5.22496

In [33]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9952614379084967

In [34]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9952614379084967

In [35]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [36]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [37]:
dat3

,0,test
0,NRS148,2
1,NRS209,2
2,NRS187,1
3,CFBREBSa116,0
4,NRS187,1
...,...,...
197,GA12,0
198,NRS209,2
199,NRS265,1
200,NRS253,1


In [38]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 362us/step - loss: 2.1068 - accuracy: 0.5000 - val_loss: 1.3162 - val_accuracy: 0.5297
Epoch 2/100
470/470 [==============================] - 0s 150us/step - loss: 0.8341 - accuracy: 0.6574 - val_loss: 0.5909 - val_accuracy: 0.7277
Epoch 3/100
470/470 [==============================] - 0s 263us/step - loss: 0.4848 - accuracy: 0.8106 - val_loss: 0.4510 - val_accuracy: 0.7970
Epoch 4/100
470/470 [==============================] - 0s 274us/step - loss: 0.4051 - accuracy: 0.8426 - val_loss: 0.3643 - val_accuracy: 0.8812
Epoch 5/100
470/470 [==============================] - 0s 239us/step - loss: 0.3350 - accuracy: 0.8851 - val_loss: 0.3231 - val_accuracy: 0.8762
Epoch 6/100
470/470 [==============================] - 0s 267us/step - loss: 0.2890 - accuracy: 0.9191 - val_loss: 0.3030 - val_accuracy: 0.9059
Epoch 7/100
470/470 [==============================] - 0s 188us/step - loss: 0.2773 

Epoch 57/100
470/470 [==============================] - 0s 121us/step - loss: 0.0311 - accuracy: 0.9957 - val_loss: 0.0447 - val_accuracy: 0.9901
Epoch 58/100
470/470 [==============================] - 0s 179us/step - loss: 0.0319 - accuracy: 0.9957 - val_loss: 0.0468 - val_accuracy: 0.9851
Epoch 59/100
470/470 [==============================] - 0s 136us/step - loss: 0.0284 - accuracy: 0.9957 - val_loss: 0.0466 - val_accuracy: 0.9851
Epoch 60/100
470/470 [==============================] - 0s 119us/step - loss: 0.0289 - accuracy: 0.9957 - val_loss: 0.0429 - val_accuracy: 0.9851
Epoch 61/100
470/470 [==============================] - 0s 124us/step - loss: 0.0293 - accuracy: 0.9957 - val_loss: 0.0440 - val_accuracy: 0.9901
Epoch 62/100
470/470 [==============================] - 0s 193us/step - loss: 0.0284 - accuracy: 0.9957 - val_loss: 0.0500 - val_accuracy: 0.9802
Epoch 63/100
470/470 [==============================] - 0s 174us/step - loss: 0.0268 - accuracy: 0.9957 - val_loss: 0.0631 -

In [63]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

202/202 [==============================] - 0s 85us/step
over-sampling test accuracy: 97.52%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([2, 2, 1, 0, 1, 2, 2, 1, 2, 1, 1, 0, 2, 1, 0, 1, 0, 1, 2, 1, 0, 0,
       0, 0, 1, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 0, 0, 1, 0, 1, 2,
       0, 1, 1, 0, 1, 0, 0, 2, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2,
       2, 2, 1, 0, 1, 0, 2, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 1,
       2, 2, 2, 1, 0, 1, 0, 1, 0, 2, 0, 1, 2, 2, 0, 2, 1, 2, 0, 1, 0, 0,
       0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 1,
       0, 0, 2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 2, 2, 1, 1, 1, 0, 2, 1, 2, 0,
       0, 2, 1, 1, 1, 2, 1, 0, 0, 2, 2, 0, 1, 2, 2, 0, 2, 1, 1, 0, 2, 0,
       1, 2, 0, 0, 0, 1, 1, 2, 1, 2, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 1, 0,
       2, 1, 1, 0])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS148,2,2
1,NRS209,2,2
2,NRS187,1,1
3,CFBREBSa116,0,0
4,NRS187,1,1
...,...,...,...
197,GA12,0,0
198,NRS209,2,2
199,NRS265,1,1
200,NRS253,1,1


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.000123,0.010629,9.892482e-01
1,0.000020,0.000197,9.997830e-01
2,0.009963,0.990030,7.961025e-06
3,0.999736,0.000264,1.595297e-07
4,0.009963,0.990030,7.961025e-06
...,...,...,...
197,0.979151,0.020849,2.218113e-10
198,0.000020,0.000197,9.997830e-01
199,0.001390,0.996783,1.827248e-03
200,0.000794,0.998547,6.587048e-04


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p003ppST.csv", index = False,
         header=None)

In [67]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 218us/step - loss: 0.0127 - accuracy: 0.9957 - val_loss: 0.0463 - val_accuracy: 0.9752
Epoch 2/100
470/470 [==============================] - 0s 210us/step - loss: 0.0153 - accuracy: 0.9979 - val_loss: 0.0384 - val_accuracy: 0.9752
Epoch 3/100
470/470 [==============================] - 0s 206us/step - loss: 0.0133 - accuracy: 0.9957 - val_loss: 0.0454 - val_accuracy: 0.9752
Epoch 4/100
470/470 [==============================] - 0s 197us/step - loss: 0.0131 - accuracy: 0.9957 - val_loss: 0.0396 - val_accuracy: 0.9752
Epoch 5/100
470/470 [==============================] - 0s 208us/step - loss: 0.0129 - accuracy: 1.0000 - val_loss: 0.0636 - val_accuracy: 0.9752
Epoch 6/100
470/470 [==============================] - 0s 208us/step - loss: 0.0161 - accuracy: 0.9957 - val_loss: 0.0482 - val_accuracy: 0.9752
Epoch 7/100
470/470 [==============================] - 0s 205us/step - loss: 0.0139 

Epoch 57/100
470/470 [==============================] - 0s 135us/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.0604 - val_accuracy: 0.9752
Epoch 58/100
470/470 [==============================] - 0s 131us/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.0507 - val_accuracy: 0.9752
Epoch 59/100
470/470 [==============================] - 0s 139us/step - loss: 0.0062 - accuracy: 0.9979 - val_loss: 0.0399 - val_accuracy: 0.9752
Epoch 60/100
470/470 [==============================] - 0s 135us/step - loss: 0.0057 - accuracy: 0.9979 - val_loss: 0.0474 - val_accuracy: 0.9752
Epoch 61/100
470/470 [==============================] - 0s 134us/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0613 - val_accuracy: 0.9752
Epoch 62/100
470/470 [==============================] - 0s 132us/step - loss: 0.0066 - accuracy: 0.9979 - val_loss: 0.0314 - val_accuracy: 0.9851
Epoch 63/100
470/470 [==============================] - 0s 118us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0682 -

In [68]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 99.87%


In [39]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [40]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [41]:
y_prob3 = df_proba3[df_proba3['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[1.22783270e-04, 1.06290110e-02, 9.89248200e-01],
       [2.00190680e-05, 1.96983950e-04, 9.99783000e-01],
       [9.96255400e-03, 9.90029500e-01, 7.96102500e-06],
       [9.99736130e-01, 2.63790540e-04, 1.59529660e-07],
       [9.96255400e-03, 9.90029500e-01, 7.96102500e-06],
       [2.00190680e-05, 1.96983950e-04, 9.99783000e-01],
       [5.12395750e-04, 1.28842060e-04, 9.99358700e-01],
       [7.92104900e-03, 9.92078840e-01, 1.16210340e-07],
       [1.22783270e-04, 1.06290110e-02, 9.89248200e-01],
       [1.38985540e-03, 9.96782800e-01, 1.82724940e-03],
       [6.71824400e-03, 9.93279040e-01, 2.67666600e-06],
       [9.98966600e-01, 1.03315410e-03, 2.15709140e-07],
       [2.00190680e-05, 1.96983950e-04, 9.99783000e-01],
       [6.07249330e-03, 9.93094300e-01, 8.33168200e-04],
       [9.99908300e-01, 9.16254800e-05, 1.36625210e-10],
       [7.92104900e-03, 9.92078840e-01, 1.16210340e-07],
       [9.99998900e-01, 1.10197580e-06, 5.12359540e-08],
       [5.37403100e-03, 9.93032

In [42]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

1.0

In [43]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

1.0

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [45]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [46]:
dat4

,0,test
0,CFBRSa04,0
1,NRS021,0
2,NRS073,0
3,NRS049,0
4,CA541,1
...,...,...
197,NRS387,1
198,SR1746,0
199,NRS148,2
200,NRS255,2


In [47]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 631us/step - loss: 3.9849 - accuracy: 0.5702 - val_loss: 2.8639 - val_accuracy: 0.5941
Epoch 2/100
470/470 [==============================] - 0s 230us/step - loss: 1.5341 - accuracy: 0.6277 - val_loss: 0.6642 - val_accuracy: 0.8168
Epoch 3/100
470/470 [==============================] - 0s 136us/step - loss: 0.5829 - accuracy: 0.7298 - val_loss: 0.3898 - val_accuracy: 0.8168
Epoch 4/100
470/470 [==============================] - 0s 144us/step - loss: 0.4166 - accuracy: 0.8404 - val_loss: 0.3577 - val_accuracy: 0.8416
Epoch 5/100
470/470 [==============================] - 0s 168us/step - loss: 0.3519 - accuracy: 0.8745 - val_loss: 0.2847 - val_accuracy: 0.8960
Epoch 6/100
470/470 [==============================] - 0s 242us/step - loss: 0.2743 - accuracy: 0.9085 - val_loss: 0.2463 - val_accuracy: 0.8960
Epoch 7/100
470/470 [==============================] - 0s 181us/step - loss: 0.2565 

Epoch 57/100
470/470 [==============================] - 0s 119us/step - loss: 0.0323 - accuracy: 0.9957 - val_loss: 0.0494 - val_accuracy: 0.9851
Epoch 58/100
470/470 [==============================] - 0s 135us/step - loss: 0.0339 - accuracy: 0.9957 - val_loss: 0.0434 - val_accuracy: 0.9901
Epoch 59/100
470/470 [==============================] - 0s 184us/step - loss: 0.0398 - accuracy: 0.9936 - val_loss: 0.0424 - val_accuracy: 0.9851
Epoch 60/100
470/470 [==============================] - 0s 126us/step - loss: 0.0497 - accuracy: 0.9894 - val_loss: 0.0531 - val_accuracy: 0.9851
Epoch 61/100
470/470 [==============================] - 0s 119us/step - loss: 0.0435 - accuracy: 0.9872 - val_loss: 0.0779 - val_accuracy: 0.9752
Epoch 62/100
470/470 [==============================] - 0s 132us/step - loss: 0.0536 - accuracy: 0.9872 - val_loss: 0.1798 - val_accuracy: 0.9703
Epoch 63/100
470/470 [==============================] - 0s 134us/step - loss: 0.0989 - accuracy: 0.9723 - val_loss: 0.0665 -

In [76]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

202/202 [==============================] - 0s 108us/step
over-sampling test accuracy: 99.01%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 1, 0, 2, 1, 2, 0,
       0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 0, 0,
       0, 1, 2, 1, 1, 0, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 0, 2, 2, 2, 0, 0,
       2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 0, 1, 1, 0, 2, 2, 0, 2,
       2, 1, 2, 0, 0, 2, 1, 1, 2, 2, 2, 1, 1, 2, 0, 2, 1, 1, 1, 2, 0, 0,
       0, 1, 2, 1, 1, 2, 0, 1, 2, 0, 0, 2, 1, 2, 1, 2, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 2, 0, 1, 1, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2, 2, 2,
       1, 2, 1, 2, 0, 2, 0, 0, 2, 1, 1, 0, 1, 0, 2, 0, 0, 0, 2, 0, 2, 1,
       2, 2, 1, 2, 1, 2, 2, 1, 0, 2, 2, 0, 0, 2, 1, 0, 0, 2, 2, 2, 0, 1,
       0, 2, 2, 1])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,CFBRSa04,0,0
1,NRS021,0,0
2,NRS073,0,0
3,NRS049,0,0
4,CA541,1,1
...,...,...,...
197,NRS387,1,1
198,SR1746,0,0
199,NRS148,2,2
200,NRS255,2,2


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.999366,0.000633,4.865112e-07
1,0.983386,0.016614,1.033543e-10
2,0.975474,0.024526,2.480035e-10
3,0.570170,0.428658,1.171963e-03
4,0.007818,0.992128,5.324918e-05
...,...,...,...
197,0.007140,0.992790,6.977263e-05
198,0.516005,0.483995,2.800119e-08
199,0.000003,0.007481,9.925161e-01
200,0.000111,0.001138,9.987515e-01


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p003ppST.csv", index = False,
         header=None)

In [85]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 298us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0184 - val_accuracy: 0.9901
Epoch 2/100
470/470 [==============================] - 0s 219us/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0281 - val_accuracy: 0.9851
Epoch 3/100
470/470 [==============================] - 0s 190us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0202 - val_accuracy: 0.9901
Epoch 4/100
470/470 [==============================] - 0s 175us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0214 - val_accuracy: 0.9901
Epoch 5/100
470/470 [==============================] - 0s 171us/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.0186 - val_accuracy: 0.9901
Epoch 6/100
470/470 [==============================] - 0s 208us/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.0229 - val_accuracy: 0.9901
Epoch 7/100
470/470 [==============================] - 0s 137us/step - loss: 0.0020 

In [86]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [48]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [49]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [50]:
y_prob4 = df_proba4[df_proba4['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[9.99366100e-01, 6.33382200e-04, 4.86511200e-07],
       [9.83386000e-01, 1.66140370e-02, 1.03354290e-10],
       [9.75474100e-01, 2.45258720e-02, 2.48003480e-10],
       [5.70169870e-01, 4.28658220e-01, 1.17196330e-03],
       [7.81840200e-03, 9.92128400e-01, 5.32491800e-05],
       [1.13398490e-03, 9.98866100e-01, 7.69132000e-12],
       [4.98751900e-04, 9.97489100e-01, 2.01220740e-03],
       [3.22916300e-03, 9.96744160e-01, 2.66253920e-05],
       [9.96499660e-01, 3.49792580e-03, 2.34041980e-06],
       [9.99869470e-01, 1.30563840e-04, 1.34077836e-11],
       [9.99535440e-01, 4.64594220e-04, 9.93066200e-10],
       [9.56812860e-01, 4.31799930e-02, 7.20720570e-06],
       [1.10811154e-04, 1.13763330e-03, 9.98751500e-01],
       [1.34385970e-08, 4.03788030e-04, 9.99596200e-01],
       [1.10811154e-04, 1.13763330e-03, 9.98751500e-01],
       [1.34385970e-08, 4.03788030e-04, 9.99596200e-01],
       [4.32178460e-05, 9.99825660e-01, 1.31092240e-04],
       [8.10826000e-01, 1.89174

In [51]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9998168200392374

In [52]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9998168200392374

In [53]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9968071543155681

In [54]:
np.std(ovos)

0.003291174950740655

In [55]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9968071543155681

In [56]:
np.std(ovrs)

0.003291174950740655

In [87]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [88]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 96.78%


In [89]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.018020066566141215


In [90]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [91]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 99.92%


In [92]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0008180433


In [57]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [58]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [59]:
dat5

,0,test
0,NRS255,2
1,NRS148,2
2,NRS209,2
3,NRS386,1
4,NRS230,0
...,...,...
197,NRS209,2
198,NRS209,2
199,SR4153,0
200,NRS255,2


In [60]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [61]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [71]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 809us/step - loss: 7.3546 - accuracy: 0.3745 - val_loss: 2.9202 - val_accuracy: 0.5099
Epoch 2/100
470/470 [==============================] - 0s 357us/step - loss: 4.8069 - accuracy: 0.5298 - val_loss: 1.8845 - val_accuracy: 0.5990
Epoch 3/100
470/470 [==============================] - 0s 187us/step - loss: 4.7142 - accuracy: 0.4830 - val_loss: 1.2099 - val_accuracy: 0.7426
Epoch 4/100
470/470 [==============================] - 0s 229us/step - loss: 3.4922 - accuracy: 0.5851 - val_loss: 1.0248 - val_accuracy: 0.6980
Epoch 5/100
470/470 [==============================] - 0s 318us/step - loss: 3.5853 - accuracy: 0.5830 - val_loss: 0.8800 - val_accuracy: 0.7277
Epoch 6/100
470/470 [==============================] - 0s 274us/step - loss: 3.1891 - accuracy: 0.6511 - val_loss: 0.8346 - val_accuracy: 0.7228
Epoch 7/100
470/470 [==============================] - 0s 457us/step - loss: 3.6344 

Epoch 57/100
470/470 [==============================] - 0s 130us/step - loss: 0.9133 - accuracy: 0.7745 - val_loss: 0.5972 - val_accuracy: 0.9455
Epoch 58/100
470/470 [==============================] - 0s 135us/step - loss: 0.8894 - accuracy: 0.7872 - val_loss: 0.3665 - val_accuracy: 0.9703
Epoch 59/100
470/470 [==============================] - 0s 146us/step - loss: 0.9349 - accuracy: 0.7660 - val_loss: 0.4248 - val_accuracy: 0.9653
Epoch 60/100
470/470 [==============================] - 0s 168us/step - loss: 0.8415 - accuracy: 0.8021 - val_loss: 0.4406 - val_accuracy: 0.9554
Epoch 61/100
470/470 [==============================] - 0s 130us/step - loss: 0.7811 - accuracy: 0.8170 - val_loss: 0.6464 - val_accuracy: 0.9406
Epoch 62/100
470/470 [==============================] - 0s 125us/step - loss: 0.9617 - accuracy: 0.7723 - val_loss: 0.3154 - val_accuracy: 0.9901
Epoch 63/100
470/470 [==============================] - 0s 133us/step - loss: 0.9368 - accuracy: 0.7809 - val_loss: 0.3706 -

In [100]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

202/202 [==============================] - 0s 100us/step
over-sampling test accuracy: 96.53%


In [73]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([2, 2, 2, 1, 0, 0, 1, 1, 2, 0, 1, 0, 2, 0, 1, 0, 0, 2, 0, 1, 1, 2,
       2, 2, 0, 0, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 0, 2, 0, 1, 1, 2, 2, 0,
       2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 1, 1, 2, 2, 0, 0, 1, 0,
       1, 2, 1, 0, 2, 0, 1, 2, 0, 2, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 1, 0, 1, 2, 1, 1, 2, 0,
       0, 2, 1, 0, 1, 2, 0, 2, 1, 2, 0, 1, 0, 2, 0, 1, 2, 0, 1, 0, 1, 1,
       2, 2, 1, 0, 1, 1, 0, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 2,
       0, 0, 1, 1, 2, 0, 2, 1, 0, 2, 1, 1, 1, 1, 0, 2, 0, 0, 0, 0, 2, 1,
       1, 1, 1, 0, 2, 0, 2, 0, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 2, 0, 2, 2,
       2, 0, 2, 2])

In [74]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS255,2,2
1,NRS148,2,2
2,NRS209,2,2
3,NRS386,1,1
4,NRS230,0,0
...,...,...,...
197,NRS209,2,2
198,NRS209,2,2
199,SR4153,0,0
200,NRS255,2,2


In [75]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [76]:
dat_proba5

,0,1,2
0,1.602222e-08,1.254396e-15,1.000000e+00
1,1.275652e-08,6.204051e-08,9.999999e-01
2,4.458716e-09,2.097634e-08,1.000000e+00
3,6.089555e-09,1.000000e+00,1.170724e-09
4,1.000000e+00,4.748846e-18,2.535741e-13
...,...,...,...
197,4.458708e-09,2.097626e-08,1.000000e+00
198,4.458708e-09,2.097626e-08,1.000000e+00
199,1.000000e+00,8.079975e-17,2.880661e-26
200,1.602225e-08,1.254401e-15,1.000000e+00


In [77]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [78]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p003ppST.csv", index = False,
         header=None)

In [104]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 205us/step - loss: 0.8577 - accuracy: 0.7723 - val_loss: 0.5023 - val_accuracy: 0.9604
Epoch 2/100
470/470 [==============================] - 0s 142us/step - loss: 0.9891 - accuracy: 0.7553 - val_loss: 0.7670 - val_accuracy: 0.9505
Epoch 3/100
470/470 [==============================] - 0s 130us/step - loss: 0.8723 - accuracy: 0.7830 - val_loss: 0.4425 - val_accuracy: 0.9703
Epoch 4/100
470/470 [==============================] - 0s 129us/step - loss: 0.7353 - accuracy: 0.8064 - val_loss: 0.4719 - val_accuracy: 0.9653
Epoch 5/100
470/470 [==============================] - 0s 128us/step - loss: 0.6944 - accuracy: 0.7979 - val_loss: 0.5076 - val_accuracy: 0.9554
Epoch 6/100
470/470 [==============================] - 0s 135us/step - loss: 0.7644 - accuracy: 0.7872 - val_loss: 0.4127 - val_accuracy: 0.9653
Epoch 7/100
470/470 [==============================] - 0s 129us/step - loss: 0.7976 

Epoch 57/100
470/470 [==============================] - 0s 137us/step - loss: 0.5472 - accuracy: 0.7936 - val_loss: 0.4639 - val_accuracy: 0.9653
Epoch 58/100
470/470 [==============================] - 0s 148us/step - loss: 0.6708 - accuracy: 0.7915 - val_loss: 0.4436 - val_accuracy: 0.9703
Epoch 59/100
470/470 [==============================] - 0s 134us/step - loss: 0.7360 - accuracy: 0.7702 - val_loss: 0.5082 - val_accuracy: 0.9703
Epoch 60/100
470/470 [==============================] - 0s 143us/step - loss: 0.6877 - accuracy: 0.8128 - val_loss: 0.3956 - val_accuracy: 0.9752
Epoch 61/100
470/470 [==============================] - 0s 144us/step - loss: 0.6689 - accuracy: 0.7915 - val_loss: 0.4728 - val_accuracy: 0.9653
Epoch 62/100
470/470 [==============================] - 0s 136us/step - loss: 0.7254 - accuracy: 0.7702 - val_loss: 0.4125 - val_accuracy: 0.9653
Epoch 63/100
470/470 [==============================] - 0s 138us/step - loss: 0.5916 - accuracy: 0.8085 - val_loss: 0.3855 -

In [105]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 78.96%


In [62]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [63]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [64]:
y_prob5 = df_proba5[df_proba5['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.60222200e-08, 1.25439590e-15, 1.00000000e+00],
       [1.27565190e-08, 6.20405100e-08, 9.99999900e-01],
       [4.45871600e-09, 2.09763420e-08, 1.00000000e+00],
       [6.08955500e-09, 1.00000000e+00, 1.17072420e-09],
       [1.00000000e+00, 4.74884640e-18, 2.53574070e-13],
       [1.00000000e+00, 1.23001175e-11, 4.37043600e-08],
       [8.11105140e-11, 1.00000000e+00, 3.30003330e-13],
       [1.68053830e-10, 1.00000000e+00, 1.59357100e-10],
       [4.45871600e-09, 2.09763420e-08, 1.00000000e+00],
       [1.00000000e+00, 5.08875830e-10, 2.96082520e-08],
       [1.68053830e-10, 1.00000000e+00, 1.59357100e-10],
       [1.00000000e+00, 4.62693750e-14, 1.40624480e-12],
       [1.60222200e-08, 1.25439590e-15, 1.00000000e+00],
       [1.00000000e+00, 8.76729300e-15, 5.49109900e-12],
       [4.54805300e-03, 9.85998150e-01, 9.45374900e-03],
       [1.00000000e+00, 1.40867750e-15, 1.78980150e-13],
       [1.00000000e+00, 2.67006440e-09, 1.44676155e-11],
       [4.45871600e-09, 2.09763

In [65]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9944271019629521

In [66]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9944271019629521

In [67]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [68]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [69]:
dat6

,0,test
0,115,1
1,NRS209,2
2,GA984,0
3,NRS187,1
4,NRS148,2
...,...,...
197,NRS253,1
198,EUH15,0
199,NRS180,1
200,NRS266,1


In [70]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [71]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [84]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 687us/step - loss: 8.4326 - accuracy: 0.4149 - val_loss: 3.5536 - val_accuracy: 0.5000
Epoch 2/100
470/470 [==============================] - 0s 224us/step - loss: 5.5203 - accuracy: 0.5170 - val_loss: 1.9669 - val_accuracy: 0.7376
Epoch 3/100
470/470 [==============================] - 0s 244us/step - loss: 4.4551 - accuracy: 0.6213 - val_loss: 1.1876 - val_accuracy: 0.8218
Epoch 4/100
470/470 [==============================] - 0s 198us/step - loss: 3.9019 - accuracy: 0.5979 - val_loss: 0.8677 - val_accuracy: 0.7178
Epoch 5/100
470/470 [==============================] - 0s 199us/step - loss: 3.5476 - accuracy: 0.5957 - val_loss: 0.7555 - val_accuracy: 0.7871
Epoch 6/100
470/470 [==============================] - 0s 330us/step - loss: 3.4033 - accuracy: 0.6362 - val_loss: 0.6909 - val_accuracy: 0.8713
Epoch 7/100
470/470 [==============================] - 0s 231us/step - loss: 3.4419 

Epoch 57/100
470/470 [==============================] - 0s 166us/step - loss: 1.5973 - accuracy: 0.7851 - val_loss: 1.0340 - val_accuracy: 0.8960
Epoch 58/100
470/470 [==============================] - 0s 134us/step - loss: 1.7951 - accuracy: 0.7426 - val_loss: 0.6791 - val_accuracy: 0.9455
Epoch 59/100
470/470 [==============================] - 0s 278us/step - loss: 1.6915 - accuracy: 0.7638 - val_loss: 0.8388 - val_accuracy: 0.9653
Epoch 60/100
470/470 [==============================] - 0s 143us/step - loss: 1.9093 - accuracy: 0.7362 - val_loss: 0.7838 - val_accuracy: 0.9554
Epoch 61/100
470/470 [==============================] - 0s 141us/step - loss: 1.7651 - accuracy: 0.7532 - val_loss: 0.9419 - val_accuracy: 0.9356
Epoch 62/100
470/470 [==============================] - 0s 167us/step - loss: 1.8136 - accuracy: 0.7468 - val_loss: 0.7847 - val_accuracy: 0.9604
Epoch 63/100
470/470 [==============================] - 0s 171us/step - loss: 1.5301 - accuracy: 0.7596 - val_loss: 0.8855 -

In [125]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

202/202 [==============================] - 0s 124us/step
over-sampling test accuracy: 96.04%


In [86]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([1, 2, 0, 1, 2, 2, 1, 1, 1, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0,
       2, 0, 2, 0, 2, 0, 0, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 0, 1,
       2, 2, 2, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0,
       1, 1, 2, 2, 0, 2, 2, 2, 0, 1, 0, 0, 2, 2, 0, 2, 2, 2, 1, 0, 1, 1,
       0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 1, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2,
       0, 0, 0, 2, 1, 0, 1, 1, 2, 0, 2, 2, 1, 1, 0, 0, 2, 1, 1, 1, 1, 2,
       1, 0, 2, 2, 1, 2, 0, 1, 2, 0, 1, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1,
       1, 2, 0, 2, 2, 0, 2, 1, 0, 2, 0, 2, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2,
       1, 0, 2, 2, 2, 0, 0, 1, 1, 0, 0, 0, 2, 2, 0, 0, 1, 1, 0, 0, 2, 1,
       0, 1, 1, 1])

In [87]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,115,1,1
1,NRS209,2,2
2,GA984,0,0
3,NRS187,1,1
4,NRS148,2,2
...,...,...,...
197,NRS253,1,1
198,EUH15,0,0
199,NRS180,1,1
200,NRS266,1,1


In [88]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [89]:
dat_proba6

,0,1,2
0,1.250444e-05,9.999866e-01,7.751040e-07
1,6.182140e-08,6.756152e-08,9.999999e-01
2,9.999980e-01,1.788405e-06,1.889782e-07
3,4.463270e-05,9.999526e-01,2.906656e-06
4,8.719623e-09,6.788134e-08,9.999999e-01
...,...,...,...
197,2.495610e-01,5.174635e-01,2.329755e-01
198,1.000000e+00,1.395472e-15,6.243795e-16
199,1.665770e-06,9.999982e-01,1.231557e-07
200,2.495610e-01,5.174635e-01,2.329755e-01


In [90]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [91]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p003ppST.csv", index = False,
         header=None)

In [129]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 209us/step - loss: 0.9105 - accuracy: 0.8064 - val_loss: 0.5378 - val_accuracy: 0.9554
Epoch 2/100
470/470 [==============================] - 0s 210us/step - loss: 1.0259 - accuracy: 0.8128 - val_loss: 0.6205 - val_accuracy: 0.9604
Epoch 3/100
470/470 [==============================] - 0s 162us/step - loss: 1.0646 - accuracy: 0.7340 - val_loss: 0.5300 - val_accuracy: 0.9653
Epoch 4/100
470/470 [==============================] - 0s 155us/step - loss: 0.9789 - accuracy: 0.7745 - val_loss: 0.4759 - val_accuracy: 0.9653
Epoch 5/100
470/470 [==============================] - 0s 159us/step - loss: 1.0393 - accuracy: 0.8021 - val_loss: 0.7389 - val_accuracy: 0.9653
Epoch 6/100
470/470 [==============================] - 0s 137us/step - loss: 0.9923 - accuracy: 0.8064 - val_loss: 0.5495 - val_accuracy: 0.9604
Epoch 7/100
470/470 [==============================] - 0s 128us/step - loss: 0.9844 

Epoch 57/100
470/470 [==============================] - 0s 207us/step - loss: 0.8362 - accuracy: 0.7979 - val_loss: 0.3841 - val_accuracy: 0.9950
Epoch 58/100
470/470 [==============================] - 0s 141us/step - loss: 0.7849 - accuracy: 0.7936 - val_loss: 0.4090 - val_accuracy: 0.9950
Epoch 59/100
470/470 [==============================] - 0s 132us/step - loss: 0.7447 - accuracy: 0.8085 - val_loss: 0.3857 - val_accuracy: 0.9950
Epoch 60/100
470/470 [==============================] - 0s 121us/step - loss: 0.8468 - accuracy: 0.7681 - val_loss: 0.4218 - val_accuracy: 0.9901
Epoch 61/100
470/470 [==============================] - 0s 126us/step - loss: 0.7680 - accuracy: 0.7979 - val_loss: 0.4205 - val_accuracy: 0.9901
Epoch 62/100
470/470 [==============================] - 0s 122us/step - loss: 0.7581 - accuracy: 0.8234 - val_loss: 0.4109 - val_accuracy: 0.9901
Epoch 63/100
470/470 [==============================] - 0s 121us/step - loss: 0.7968 - accuracy: 0.7660 - val_loss: 0.3852 -

In [130]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 79.52%


In [72]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [73]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [74]:
y_prob6 = df_proba6[df_proba6['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[1.25044390e-05, 9.99986650e-01, 7.75104000e-07],
       [6.18213960e-08, 6.75615150e-08, 9.99999900e-01],
       [9.99998000e-01, 1.78840510e-06, 1.88978180e-07],
       [4.46327000e-05, 9.99952550e-01, 2.90665600e-06],
       [8.71962300e-09, 6.78813400e-08, 9.99999900e-01],
       [6.18213960e-08, 6.75615150e-08, 9.99999900e-01],
       [1.71000660e-07, 9.99999900e-01, 7.98898000e-09],
       [3.39365070e-05, 9.99965900e-01, 8.48425600e-08],
       [1.10746770e-06, 9.99998800e-01, 7.54087700e-08],
       [1.00000000e+00, 5.10242600e-09, 3.64943900e-10],
       [6.18213960e-08, 6.75615150e-08, 9.99999900e-01],
       [2.27158640e-07, 9.99999760e-01, 1.55337300e-09],
       [6.18213960e-08, 6.75615150e-08, 9.99999900e-01],
       [3.39365070e-05, 9.99965900e-01, 8.48425600e-08],
       [1.00000000e+00, 9.29824900e-11, 5.82753640e-11],
       [1.00000000e+00, 8.23512540e-10, 8.64572160e-11],
       [9.99548260e-01, 4.51167580e-04, 5.44053650e-07],
       [6.18213960e-08, 6.75615

In [75]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9974728752750409

In [76]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9974728752750409

In [77]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [78]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [79]:
dat7

,0,test
0,NRS260,0
1,NRS148,2
2,NRS205,1
3,NRS064,1
4,NRS209,2
...,...,...
197,NRS255,2
198,GA53649,0
199,NRS209,2
200,NRS210,0


In [80]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [96]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [97]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 657us/step - loss: 6.3820 - accuracy: 0.4234 - val_loss: 3.3629 - val_accuracy: 0.5842
Epoch 2/100
470/470 [==============================] - 0s 212us/step - loss: 4.5751 - accuracy: 0.5021 - val_loss: 2.2508 - val_accuracy: 0.5594
Epoch 3/100
470/470 [==============================] - 0s 264us/step - loss: 3.8921 - accuracy: 0.5553 - val_loss: 1.6998 - val_accuracy: 0.5297
Epoch 4/100
470/470 [==============================] - 0s 197us/step - loss: 4.3578 - accuracy: 0.4787 - val_loss: 1.3396 - val_accuracy: 0.6683
Epoch 5/100
470/470 [==============================] - 0s 174us/step - loss: 4.4661 - accuracy: 0.4702 - val_loss: 1.1898 - val_accuracy: 0.6089
Epoch 6/100
470/470 [==============================] - 0s 215us/step - loss: 4.0276 - accuracy: 0.5106 - val_loss: 1.0072 - val_accuracy: 0.6535
Epoch 7/100
470/470 [==============================] - 0s 229us/step - loss: 3.8183 

Epoch 57/100
470/470 [==============================] - 0s 146us/step - loss: 2.2022 - accuracy: 0.7702 - val_loss: 0.8341 - val_accuracy: 0.9455
Epoch 58/100
470/470 [==============================] - 0s 140us/step - loss: 1.9290 - accuracy: 0.8000 - val_loss: 0.8023 - val_accuracy: 0.9356
Epoch 59/100
470/470 [==============================] - 0s 141us/step - loss: 1.8256 - accuracy: 0.8043 - val_loss: 0.7745 - val_accuracy: 0.9604
Epoch 60/100
470/470 [==============================] - 0s 131us/step - loss: 2.0107 - accuracy: 0.7851 - val_loss: 0.7809 - val_accuracy: 0.9455
Epoch 61/100
470/470 [==============================] - 0s 134us/step - loss: 1.8318 - accuracy: 0.7915 - val_loss: 0.7610 - val_accuracy: 0.9455
Epoch 62/100
470/470 [==============================] - 0s 171us/step - loss: 1.8657 - accuracy: 0.7979 - val_loss: 0.7344 - val_accuracy: 0.9604
Epoch 63/100
470/470 [==============================] - 0s 198us/step - loss: 2.0620 - accuracy: 0.7404 - val_loss: 0.7937 -

In [138]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

202/202 [==============================] - 0s 99us/step
over-sampling test accuracy: 93.07%


In [99]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([0, 2, 2, 1, 2, 1, 1, 1, 2, 0, 0, 1, 2, 1, 2, 0, 2, 0, 1, 0, 0, 1,
       2, 0, 1, 2, 2, 2, 2, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 0, 2, 0, 1, 1,
       0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 0, 0, 2, 0, 1,
       2, 2, 2, 0, 1, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1,
       0, 2, 1, 2, 0, 2, 2, 1, 2, 0, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 0, 1, 1, 1, 2, 2, 1, 0,
       1, 1, 0, 1, 2, 2, 2, 0, 2, 1, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2,
       1, 1, 1, 2, 1, 0, 1, 2, 2, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 2,
       0, 1, 2, 0, 0, 1, 2, 1, 0, 1, 0, 2, 2, 1, 1, 2, 2, 1, 2, 0, 2, 2,
       0, 2, 0, 0])

In [100]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS260,0,0
1,NRS148,2,2
2,NRS205,1,2
3,NRS064,1,1
4,NRS209,2,2
...,...,...,...
197,NRS255,2,2
198,GA53649,0,0
199,NRS209,2,2
200,NRS210,0,0


In [101]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [102]:
dat_proba7

,0,1,2
0,9.498861e-01,4.973201e-02,3.818146e-04
1,5.827860e-07,3.119154e-06,9.999963e-01
2,3.094362e-04,9.128791e-04,9.987777e-01
3,3.611043e-11,1.000000e+00,5.426886e-09
4,3.801006e-08,3.132352e-08,9.999999e-01
...,...,...,...
197,1.471449e-01,2.311823e-01,6.216729e-01
198,9.999999e-01,1.012713e-07,2.161844e-08
199,3.801013e-08,3.132352e-08,9.999999e-01
200,9.790896e-01,2.091015e-02,2.395272e-07


In [103]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [104]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p003ppST.csv", index = False,
         header=None)

In [142]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 200us/step - loss: 1.0190 - accuracy: 0.7851 - val_loss: 0.7314 - val_accuracy: 0.9307
Epoch 2/100
470/470 [==============================] - 0s 140us/step - loss: 0.8814 - accuracy: 0.8043 - val_loss: 0.5911 - val_accuracy: 0.9307
Epoch 3/100
470/470 [==============================] - 0s 131us/step - loss: 1.0980 - accuracy: 0.7979 - val_loss: 0.7353 - val_accuracy: 0.9307
Epoch 4/100
470/470 [==============================] - 0s 133us/step - loss: 1.0050 - accuracy: 0.7596 - val_loss: 0.6711 - val_accuracy: 0.9307
Epoch 5/100
470/470 [==============================] - 0s 125us/step - loss: 0.8658 - accuracy: 0.7766 - val_loss: 0.6054 - val_accuracy: 0.9307
Epoch 6/100
470/470 [==============================] - 0s 123us/step - loss: 0.8575 - accuracy: 0.7894 - val_loss: 0.6344 - val_accuracy: 0.9208
Epoch 7/100
470/470 [==============================] - 0s 133us/step - loss: 0.9313 

Epoch 57/100
470/470 [==============================] - 0s 165us/step - loss: 0.7476 - accuracy: 0.8064 - val_loss: 0.4392 - val_accuracy: 0.9703
Epoch 58/100
470/470 [==============================] - 0s 466us/step - loss: 0.6842 - accuracy: 0.8149 - val_loss: 0.4946 - val_accuracy: 0.9554
Epoch 59/100
470/470 [==============================] - 0s 183us/step - loss: 0.7550 - accuracy: 0.7872 - val_loss: 0.5006 - val_accuracy: 0.9604
Epoch 60/100
470/470 [==============================] - 0s 170us/step - loss: 0.8078 - accuracy: 0.7681 - val_loss: 0.4707 - val_accuracy: 0.9653
Epoch 61/100
470/470 [==============================] - 0s 143us/step - loss: 0.8197 - accuracy: 0.7766 - val_loss: 0.4902 - val_accuracy: 0.9653
Epoch 62/100
470/470 [==============================] - 0s 133us/step - loss: 0.7267 - accuracy: 0.7851 - val_loss: 0.4780 - val_accuracy: 0.9604
Epoch 63/100
470/470 [==============================] - 0s 139us/step - loss: 0.7281 - accuracy: 0.7936 - val_loss: 0.4597 -

In [143]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 78.60%


In [81]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [82]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [83]:
y_prob7 = df_proba7[df_proba7['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[9.49886140e-01, 4.97320070e-02, 3.81814580e-04],
       [5.82786000e-07, 3.11915370e-06, 9.99996300e-01],
       [3.09436200e-04, 9.12879100e-04, 9.98777700e-01],
       [3.61104300e-11, 1.00000000e+00, 5.42688560e-09],
       [3.80100600e-08, 3.13235230e-08, 9.99999900e-01],
       [3.89797400e-05, 9.99961000e-01, 4.71664020e-08],
       [9.99025800e-04, 9.98982500e-01, 1.85122120e-05],
       [3.89797400e-05, 9.99961000e-01, 4.71664020e-08],
       [3.80100600e-08, 3.13235230e-08, 9.99999900e-01],
       [1.00000000e+00, 5.07421300e-09, 5.74017500e-08],
       [1.00000000e+00, 1.04493490e-08, 6.94282350e-10],
       [5.43320000e-04, 9.99450600e-01, 6.02986600e-06],
       [3.80100600e-08, 3.13235230e-08, 9.99999900e-01],
       [1.10361455e-07, 9.99999640e-01, 2.05889580e-07],
       [1.47144910e-01, 2.31182260e-01, 6.21672870e-01],
       [1.00000000e+00, 4.78651750e-12, 1.24605050e-10],
       [5.82786000e-07, 3.11915370e-06, 9.99996300e-01],
       [9.99851800e-01, 1.48200

In [84]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9891847949793516

In [85]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9891847949793516

In [86]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [87]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [88]:
dat8

,0,test
0,CA541,1
1,SR3585,0
2,NRS232,1
3,NRS148,2
4,NRS180,1
...,...,...
197,NRS209,2
198,NRS035,1
199,506,0
200,SR2091,0


In [89]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [109]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 793us/step - loss: 6.2471 - accuracy: 0.4851 - val_loss: 4.2075 - val_accuracy: 0.5545
Epoch 2/100
470/470 [==============================] - 0s 240us/step - loss: 4.8005 - accuracy: 0.5936 - val_loss: 3.1090 - val_accuracy: 0.6733
Epoch 3/100
470/470 [==============================] - 0s 333us/step - loss: 3.7839 - accuracy: 0.5936 - val_loss: 2.0792 - val_accuracy: 0.6040
Epoch 4/100
470/470 [==============================] - 0s 284us/step - loss: 3.5625 - accuracy: 0.5596 - val_loss: 1.6336 - val_accuracy: 0.6139
Epoch 5/100
470/470 [==============================] - 0s 207us/step - loss: 3.3491 - accuracy: 0.5936 - val_loss: 1.1513 - val_accuracy: 0.6782
Epoch 6/100
470/470 [==============================] - 0s 138us/step - loss: 3.3176 - accuracy: 0.6149 - val_loss: 1.0274 - val_accuracy: 0.6238
Epoch 7/100
470/470 [==============================] - 0s 407us/step - loss: 3.1790 

Epoch 57/100
470/470 [==============================] - 0s 236us/step - loss: 1.7996 - accuracy: 0.7553 - val_loss: 0.8988 - val_accuracy: 0.9554
Epoch 58/100
470/470 [==============================] - 0s 201us/step - loss: 1.5779 - accuracy: 0.7851 - val_loss: 0.8779 - val_accuracy: 0.9653
Epoch 59/100
470/470 [==============================] - 0s 196us/step - loss: 1.8773 - accuracy: 0.7723 - val_loss: 0.8703 - val_accuracy: 0.9505
Epoch 60/100
470/470 [==============================] - 0s 216us/step - loss: 1.5385 - accuracy: 0.8000 - val_loss: 0.6631 - val_accuracy: 0.9505
Epoch 61/100
470/470 [==============================] - 0s 310us/step - loss: 1.7927 - accuracy: 0.7936 - val_loss: 0.7436 - val_accuracy: 0.9653
Epoch 62/100
470/470 [==============================] - 0s 132us/step - loss: 1.7163 - accuracy: 0.8021 - val_loss: 0.8057 - val_accuracy: 0.9653
Epoch 63/100
470/470 [==============================] - 0s 167us/step - loss: 1.5865 - accuracy: 0.7872 - val_loss: 0.9997 -

In [153]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

202/202 [==============================] - 0s 122us/step
over-sampling test accuracy: 96.04%


In [112]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([1, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 1, 1, 1, 0, 0, 2,
       1, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 1, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 0, 2, 2, 1, 1, 1, 0, 0, 2, 0, 2, 0, 1, 2, 0, 2, 1, 0, 2,
       0, 1, 0, 2, 2, 0, 2, 1, 0, 2, 2, 2, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 2, 2, 0, 1, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 0, 1,
       1, 2, 2, 2, 2, 1, 0, 1, 1, 2, 0, 0, 0, 0, 0, 1, 1, 2, 2, 1, 2, 0,
       0, 2, 1, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 0, 1, 0, 1, 2, 1, 0, 0, 2,
       1, 0, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 0, 0, 0,
       2, 1, 2, 0, 1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2, 2, 2,
       1, 0, 0, 2])

In [113]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,CA541,1,1
1,SR3585,0,0
2,NRS232,1,1
3,NRS148,2,2
4,NRS180,1,1
...,...,...,...
197,NRS209,2,2
198,NRS035,1,1
199,506,0,0
200,SR2091,0,0


In [114]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [115]:
dat_proba8

,0,1,2
0,4.991661e-03,9.936008e-01,1.407441e-03
1,1.000000e+00,4.557761e-13,1.481728e-12
2,2.490590e-07,9.999998e-01,5.866913e-08
3,3.942835e-08,2.896774e-07,9.999996e-01
4,1.009407e-07,9.999999e-01,3.121069e-08
...,...,...,...
197,6.620966e-08,7.176374e-08,9.999999e-01
198,2.811108e-06,9.999971e-01,4.550159e-08
199,1.000000e+00,2.385389e-14,5.528073e-14
200,9.998230e-01,1.704668e-04,6.498075e-06


In [116]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [117]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p003ppST.csv", index = False,
         header=None)

In [157]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 470 samples, validate on 202 samples
Epoch 1/100
470/470 [==============================] - 0s 219us/step - loss: 1.0719 - accuracy: 0.8383 - val_loss: 0.5484 - val_accuracy: 0.9752
Epoch 2/100
470/470 [==============================] - 0s 219us/step - loss: 1.0321 - accuracy: 0.8213 - val_loss: 0.5543 - val_accuracy: 0.9752
Epoch 3/100
470/470 [==============================] - 0s 183us/step - loss: 1.4766 - accuracy: 0.7830 - val_loss: 0.6773 - val_accuracy: 0.9752
Epoch 4/100
470/470 [==============================] - 0s 196us/step - loss: 1.1797 - accuracy: 0.7915 - val_loss: 0.5983 - val_accuracy: 0.9752
Epoch 5/100
470/470 [==============================] - 0s 175us/step - loss: 1.2408 - accuracy: 0.7894 - val_loss: 0.6757 - val_accuracy: 0.9752
Epoch 6/100
470/470 [==============================] - 0s 158us/step - loss: 1.3575 - accuracy: 0.7638 - val_loss: 0.6139 - val_accuracy: 0.9752
Epoch 7/100
470/470 [==============================] - 0s 185us/step - loss: 1.1229 

Epoch 57/100
470/470 [==============================] - 0s 235us/step - loss: 1.0470 - accuracy: 0.8149 - val_loss: 0.4288 - val_accuracy: 0.9703
Epoch 58/100
470/470 [==============================] - 0s 213us/step - loss: 0.9319 - accuracy: 0.7894 - val_loss: 0.4286 - val_accuracy: 0.9752
Epoch 59/100
470/470 [==============================] - 0s 235us/step - loss: 1.0923 - accuracy: 0.7809 - val_loss: 0.4331 - val_accuracy: 0.9752
Epoch 60/100
470/470 [==============================] - 0s 235us/step - loss: 0.8219 - accuracy: 0.8340 - val_loss: 0.4139 - val_accuracy: 0.9752
Epoch 61/100
470/470 [==============================] - 0s 213us/step - loss: 0.8618 - accuracy: 0.8298 - val_loss: 0.4557 - val_accuracy: 0.9752
Epoch 62/100
470/470 [==============================] - 0s 245us/step - loss: 0.7571 - accuracy: 0.8255 - val_loss: 0.4303 - val_accuracy: 0.9703
Epoch 63/100
470/470 [==============================] - 0s 251us/step - loss: 0.9343 - accuracy: 0.8255 - val_loss: 0.4232 -

In [158]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 79.41%


In [90]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [91]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [92]:
y_prob8 = df_proba8[df_proba8['phage']=='p003ppresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[4.99166130e-03, 9.93600800e-01, 1.40744110e-03],
       [1.00000000e+00, 4.55776100e-13, 1.48172820e-12],
       [2.49058960e-07, 9.99999760e-01, 5.86691300e-08],
       [3.94283470e-08, 2.89677360e-07, 9.99999640e-01],
       [1.00940724e-07, 9.99999900e-01, 3.12106900e-08],
       [3.94283470e-08, 2.89677360e-07, 9.99999640e-01],
       [1.39685300e-01, 2.17175500e-01, 6.43139240e-01],
       [2.17160500e-05, 8.48488340e-05, 9.99893400e-01],
       [1.38310700e-09, 1.00000000e+00, 8.87733500e-09],
       [4.45697970e-03, 9.94991840e-01, 5.51207400e-04],
       [2.78522840e-08, 1.00000000e+00, 1.67346470e-08],
       [5.03987800e-04, 9.99466960e-01, 2.91018460e-05],
       [4.98125630e-08, 1.00000000e+00, 1.12339960e-08],
       [6.62096600e-08, 7.17637400e-08, 9.99999900e-01],
       [1.00000000e+00, 2.99266930e-13, 3.16722530e-13],
       [3.94283470e-08, 2.89677360e-07, 9.99999640e-01],
       [3.52401630e-08, 1.00000000e+00, 4.19378240e-08],
       [5.03987800e-04, 9.99466

In [93]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.9808408935713588

In [94]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.9808408935713588

In [95]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9904814164471759

In [96]:
np.std(ovos2)

0.00630614460825933

In [97]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9904814164471759

In [98]:
np.std(ovrs2)

0.00630614460825933

In [159]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [160]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 95.42%


In [161]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.013725901381031714


In [162]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [163]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 79.12%


In [164]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.0036786376
